### ■ 問1　適合度検定

##### 手計算のため省略

### ■ 問2　Pythonを使って適合度検定

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import chi2

# 観測度数
observed = np.array([44, 32, 68, 54, 46, 56])

# 回数
n = observed.sum()

# 期待度数（均等 → 300回 / 6 = 50）
expected = np.array([n / 6] * 6)

# -------------------------
# 1. クロス集計表
# -------------------------
df = pd.DataFrame({
    "目": [1, 2, 3, 4, 5, 6],
    "観測度数": observed,
    "期待度数": expected
})

print("【1. 観測度数と期待度数の表】")
print(df, "\n")

# -------------------------
# 2. (観測 - 期待)^2
# -------------------------
diff_sq = (observed - expected) ** 2
df["差の二乗値"] = diff_sq

print("【2. (観測 - 期待)^2】")
print(df[["目", "差の二乗値"]], "\n")

# -------------------------
# 3. (観測 - 期待)^2 / 期待
# -------------------------
chi_terms = diff_sq / expected
df["(差^2)/期待"] = chi_terms

print("【3. (観測 - 期待)^2 / 期待】")
print(df[["目", "(差^2)/期待"]], "\n")

# -------------------------
# 4. χ^2値の計算
# -------------------------
chi2_value = chi_terms.sum()
print(f"【4. χ²値】 χ² = {chi2_value:.4f}\n")

# -------------------------
# 5. 棄却域と検定
# -------------------------
df_degree = len(observed) - 1  # 自由度 = 6 − 1 = 5
chi2_critical = chi2.ppf(0.95, df_degree)

print("【5. 有意水準5%での検定】")
print(f"自由度 df = {df_degree}")
print(f"棄却域の臨界値 χ²(0.95, df=5) = {chi2_critical:.4f}")
print(f"計算した χ²値 = {chi2_value:.4f}")

if chi2_value > chi2_critical:
    print("\n👉 結果：帰無仮説 H0 を棄却する（ズレがあると考えられる）")
else:
    print("\n👉 結果：帰無仮説 H0 を棄却できない（ズレはあるとは言えない）")

【1. 観測度数と期待度数の表】
   目  観測度数  期待度数
0  1    44  50.0
1  2    32  50.0
2  3    68  50.0
3  4    54  50.0
4  5    46  50.0
5  6    56  50.0 

【2. (観測 - 期待)^2】
   目  差の二乗値
0  1   36.0
1  2  324.0
2  3  324.0
3  4   16.0
4  5   16.0
5  6   36.0 

【3. (観測 - 期待)^2 / 期待】
   目  (差^2)/期待
0  1      0.72
1  2      6.48
2  3      6.48
3  4      0.32
4  5      0.32
5  6      0.72 

【4. χ²値】 χ² = 15.0400

【5. 有意水準5%での検定】
自由度 df = 5
棄却域の臨界値 χ²(0.95, df=5) = 11.0705
計算した χ²値 = 15.0400

👉 結果：帰無仮説 H0 を棄却する（ズレがあると考えられる）


### ■ 問3　独立性の検定

##### 手計算のため省略

### ■ 問4　Pythonを使って独立性の検定

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import chi2

# 観測データ（良品・不良品）
data = {
    "良品": [120, 240, 150, 190],
    "不良品": [24, 32, 10, 34],
}
index = ["HB", "B", "2B", "3B"]
df = pd.DataFrame(data, index=index)

# 1. 各行の小計（右側の列）
df["小計"] = df.sum(axis=1)

# 2. 各列の小計（下の行）
df.loc["小計"] = df.sum(axis=0)

print("【1〜3. 観測度数のクロス集計表】")
print(df)
print()

# 3. 全体合計
total = df.loc["小計", "小計"]

# 4. 期待度数のクロス集計表（小計行・小計列は除く）
observed = df.iloc[0:4, 0:2]  # 良品・不良品 × 4カテゴリ
row_sums = observed.sum(axis=1).values.reshape(-1, 1)
col_sums = observed.sum(axis=0).values.reshape(1, -1)

expected = (row_sums @ col_sums) / total
expected_df = pd.DataFrame(expected, index=index, columns=["良品", "不良品"])

print("【4. 期待度数のクロス集計表】")
print(expected_df)
print()

# 5. χ²値の計算
chi2_value = ((observed - expected) ** 2 / expected).sum().sum()

# 自由度 (行数−1)×(列数−1)
df_degree = (observed.shape[0] - 1) * (observed.shape[1] - 1)

# 臨界値
critical_value = chi2.ppf(0.95, df_degree)

print("【5. χ²値と臨界値】")
print(f"χ²値: {chi2_value:.3f}")
print(f"臨界値(自由度 {df_degree}, 有意水準 5%): {critical_value:.3f}")
print()

# 6. 仮説検定（独立性の検定）
print("【6. 仮説検定の結論】")
print("帰無仮説H0: カテゴリデータは独立である")
print("対立仮説H1: カテゴリデータは独立ではない\n")

if chi2_value > critical_value:
    print("⇒ χ²値 > 臨界値 → 帰無仮説H0を棄却！")
    print("⇒ 良品/不良品と鉛筆の濃さには **関連がある（独立ではない）** といえる。")
else:
    print("⇒ χ²値 ≤ 臨界値 → 帰無仮説H0を棄却できない。")
    print("⇒ 良品/不良品と鉛筆の濃さは **独立とみなせる（関連があるとはいえない）**。")


【1〜3. 観測度数のクロス集計表】
     良品  不良品   小計
HB  120   24  144
B   240   32  272
2B  150   10  160
3B  190   34  224
小計  700  100  800

【4. 期待度数のクロス集計表】
       良品   不良品
HB  126.0  18.0
B   238.0  34.0
2B  140.0  20.0
3B  196.0  28.0

【5. χ²値と臨界値】
χ²値: 9.604
臨界値(自由度 3, 有意水準 5%): 7.815

【6. 仮説検定の結論】
帰無仮説H0: カテゴリデータは独立である
対立仮説H1: カテゴリデータは独立ではない

⇒ χ²値 > 臨界値 → 帰無仮説H0を棄却！
⇒ 良品/不良品と鉛筆の濃さには **関連がある（独立ではない）** といえる。


### ■ 問5　一元配置分散分析

##### 手計算のため省略

### ■ 問6　pythonを用いて一元配置分散分析

In [12]:
import numpy as np
import pandas as pd
from scipy.stats import f

# データ（装置 A, B, C, D 各3個）
data = {
    "A": [9, 9, 12],
    "B": [13, 11, 12],
    "C": [15, 14, 16],
    "D": [11, 13, 15]
}

df = pd.DataFrame(data)

# 1. 製造されたネジの強度の平均値（全体平均）
grand_mean = df.values.flatten().mean()
print("【1. 全体平均】")
print(grand_mean)
print()

# 2. 装置ごとの平均値
group_means = df.mean()
print("【2. 装置ごとの平均値】")
print(group_means)
print()

# -------------------------
# 3.平方和の計算
# -------------------------

# 総平方和 SST = Σ(x - 全体平均)^2
SST = ((df - grand_mean)**2).sum().sum()

# 群間平方和 SSB = Σn_i (群平均 - 全体平均)^2
n = df.shape[0]  # 各群のサンプル数 = 3
SSB = sum(n * (group_means[g] - grand_mean)**2 for g in group_means.index)

# 群内平方和 SSE = SST - SSB
SSE = SST - SSB

print("【3. 平方和】")
print(f"総平方和 SST: {SST:.3f}")
print(f"群間平方和 SSB: {SSB:.3f}")
print(f"群内平方和 SSE: {SSE:.3f}")
print()

# -------------------------
# 4. 自由度
# -------------------------

k = df.shape[1]     # 群数 = 4
N = df.size         # 全データ数 = 12

df_total = N - 1         # 全体の自由度
df_between = k - 1       # 群間の自由度
df_within = N - k        # 群内の自由度

print("【4. 自由度】")
print(f"全体 df_total: {df_total}")
print(f"群間 df_between: {df_between}")
print(f"群内 df_within: {df_within}")
print()

# -------------------------
# 5. 平均平方
# -------------------------

MSB = SSB / df_between  # 群間平均平方
MSE = SSE / df_within   # 群内平均平方

print("【5. 平均平方】")
print(f"群間平均平方 MSB: {MSB:.3f}")
print(f"群内平均平方 MSE: {MSE:.3f}")
print()

# -------------------------
# 6. F値の計算
# -------------------------

F_value = MSB / MSE
print("【6. F値】")
print(F_value)
print()

# -------------------------
# 7. 仮説検定（有意水準5%）
# -------------------------

critical_value = f.ppf(0.95, df_between, df_within)

print("【7. 仮説検定】")
print(f"臨界値 F(0.05, {df_between}, {df_within}): {critical_value:.3f}")

if F_value > critical_value:
    print("⇒ F値 > 臨界値 → 帰無仮説 H0 を棄却")
    print("⇒ 装置ごとにネジ強度に **有意な差がある** と判断できる")
else:
    print("⇒ F値 ≤ 臨界値 → 帰無仮説 H0 を棄却できない")
    print("⇒ 装置ごとにネジ強度の **有意な差は認められない**")



【1. 全体平均】
12.5

【2. 装置ごとの平均値】
A    10.0
B    12.0
C    15.0
D    13.0
dtype: float64

【3. 平方和】
総平方和 SST: 57.000
群間平方和 SSB: 39.000
群内平方和 SSE: 18.000

【4. 自由度】
全体 df_total: 11
群間 df_between: 3
群内 df_within: 8

【5. 平均平方】
群間平均平方 MSB: 13.000
群内平均平方 MSE: 2.250

【6. F値】
5.777777777777778

【7. 仮説検定】
臨界値 F(0.05, 3, 8): 4.066
⇒ F値 > 臨界値 → 帰無仮説 H0 を棄却
⇒ 装置ごとにネジ強度に **有意な差がある** と判断できる


### ■ 問7　実験計画法